# MSEE UQ short course:  The $\texttt{UQpy}$ library

Application of Inference using the $\texttt{UQpy}$ module $\texttt{Inference}$. 

Detailed instructions on how to use this module can be found in the $\texttt{UQpy}$ documentation.

https://uqpyproject.readthedocs.io/en/latest/inference_doc.html

In [ ]:
import numpy as np

# Exercise 1 

Use **information-theoretic** inference to select between regression models. Consider the simple model:

$$y=m(\theta) + \epsilon$$

where $m(\cdot)$ is our numerical model that runs with $\texttt{RunModel}$ and $\epsilon$ is  noise. We have 50 synthetic measurements of $y$ using one of the following model:
    
1. $m_0(\theta)=\theta_{0} x$ (linear)
2. $m_1(\boldsymbol{\theta})=\theta_{0} x + \theta_{1} x^{2}$ (quadratic)
3. $m_2(\boldsymbol{\theta})=\theta_{0} x + \theta_{1} x^{2} + \theta_{2} x^{3}$ (cubic)

where $\theta_i$, $i=0,1, 2$ are the parameters of the model. We have also added some Gaussian noise ($\sim \mathcal{N}(0, 1)$) in the measurements which is considered to be homogeneous. You are given the file $\texttt{'data_ex1a.txt'}$ which contains the data. Select a statistical model from the set of three candidate models, given the data, using the $AIC_c$ information-theoretic criterion. You are provided with the file $\texttt{pfn.py}$ that contains the three models, written in $\texttt{Python}$.

### Step 1:

Read the data from the file.

In [ ]:
# Solution 1
data_ex1 = np.loadtxt('data_ex1a.txt')

### Step 2:

Import the necessary libraries:

1. From the $\texttt{UQpy.Inference}$ module.
- $\texttt{InferenceModel}$
- $\texttt{InfoModelSelection}$

2. From $\texttt{UQpy.RunModel}$ module import $\texttt{RunModel}$.

In [ ]:
# Solution
from UQpy.Inference import InferenceModel, InfoModelSelection
from UQpy.RunModel import RunModel

### Step 3:

Create instances of the $\texttt{RunModel}$ class for the three models.

In [ ]:
# Solution

model1 = RunModel(model_script='pfn.py', model_object_name='model_linear', vec=False,var_names=['theta_0'])
model2 = RunModel(model_script='pfn1.py', model_object_name='model_quadratic', vec=False,var_names=['theta_0','theta_1'])
model3 = RunModel(model_script='pfn2.py', model_object_name='model_cubic', vec=False,var_names=['theta_0','theta_1','theta_2'])

### Step 4:

Create instances of the $\texttt{InferenceModel}$ class for the three models.

In [ ]:
# Solution
M1 = InferenceModel(runmodel_object=model1, nparams=1, name='model_linear')
M2 = InferenceModel(runmodel_object=model2, nparams=2, name='model_quadratic')
M3 = InferenceModel(runmodel_object=model3, nparams=3, name='model_cubic')

### Step 5

Perform model selection. Create an instance of $\texttt{InfoModelSelection}$ class, employs information-theoretic criteria for model selection, using the $AIC_c$ criterion. Use the **Nelder-Mead** minimization algorithm for Maximum likelihood estimation of all candidate models.


For each model print its  probability to generate the measurements.

In [ ]:
# Solution
selector = InfoModelSelection(candidate_models=[M1,M2,M3], data=data_ex1, criterion='AICc', method=['nelder-mead']*3)
selector.run(nopt=1)
selector.sort_models()
print("Sorted models: ",[m.name for m in selector.candidate_models] )
print("Values of model probabilities: ", selector.probabilities)

### Activities

Use information-theoretic inference to select between probability models. In this case we consider the contact sphere model where randomness is assummed only in the parameter $\texttt{k}$.You are provided with file $\texttt{'data_ex1b.txt'}$ that contains synthetic measurements for the parameter.  Use model selection with $BIC$ criterion to identify the probability model that best describes the data. The set of candidate models consists of the following probability models: [normal, lognormal, uniform, chisquare, beta, Levy, inverse Gauss, exponential, Beta].

In [ ]:
# Solution
data_ex1b = np.loadtxt('data_ex1b.txt')

# Exercise 2 

Use  **Bayesian model** selection to choose between regression models. Consider the problem of Exercise 1. In this case, each candidate model $m_i$ is given a prior probability $P(m_i) = 1/3$. For each model, our prior belief about its parameters $\theta_i$ is that they are normally distributed as: 
- $m_0(\theta)$: $\theta_0\sim\mathcal{N}(0,10)$.
- $m_1(\boldsymbol{\theta})$: $\theta_0\sim\mathcal{N}(0,1)$, $\theta_1\sim\mathcal{N}(0,1)$
- $m_2(\boldsymbol{\theta})$:  $\theta_0\sim\mathcal{N}(0,1)$, $\theta_1\sim\mathcal{N}(0,2)$, $\theta_2\sim\mathcal{N}(0,0.25)$

### Step 1:

Import the necessary libraries

1. From the $\texttt{UQpy.Inference}$ module import:
- $\texttt{MLEstimation}$
- $\texttt{BayesModelSelection}$ 

2. From the $\texttt{UQpy.Distributions}$ module import:
- $\texttt{Normal}$
- $\texttt{JoinInd}$ 

In [ ]:
# Solution
from UQpy.Inference import MLEstimation, BayesModelSelection
from UQpy.Distributions import Normal, JointInd

### Step 2:

Create instances of the $\texttt{RunModel}$ class for the three models.

In [ ]:
# Solution
runmodel4 = RunModel(model_script='pfn.py', model_object_name='model_linear', vec=False,var_names=['theta_0'])
runmodel5 = RunModel(model_script='pfn1.py', model_object_name='model_quadratic', vec=False,var_names=['theta_0','theta_1'])
runmodel6 = RunModel(model_script='pfn2.py', model_object_name='model_cubic', vec=False,var_names=['theta_0','theta_1','theta_2'])

### Step 3

For each model $m_i(\boldsymbol{\theta})$ create a joint distribution object for its parameters $\boldsymbol{\theta}$ that represents our prior belief.

In [ ]:
# Solution
from UQpy.Distributions import JointInd
prior1=Normal()
prior2=JointInd(marginals=[Normal(), Normal()])
prior3=JointInd(marginals=[Normal(), Normal(), Normal()])

### Step 4

Create instances of the $\texttt{InferenceModel}$ class for the three models. In this case, for each model model you should also provide the $\texttt{error}$\_$\texttt{covariance}$ as $\texttt{np.ones(50)}$ (Why?).

In [ ]:
# Solution
model_n_params = [1, 2, 3]
model1 = InferenceModel(nparams=1, runmodel_object=runmodel4, prior=prior1,
                       error_covariance=np.ones(50), name='model_linear')
model2 = InferenceModel(nparams=2, runmodel_object=runmodel5, prior=prior2,
                       error_covariance=np.ones(50), name='model_quadratic')
model3 = InferenceModel(nparams=3, runmodel_object=runmodel6, prior=prior3,
                       error_covariance=np.ones(50), name='model_cubic')


### Step 5

Create an instance of the $\texttt{BayesModelSelection}$ class. To this end, you will need to utilize MCMC sampling methods:

1. Import the necessary libraries. From the $\texttt{UQpy.SampleMethods}$ module import $\texttt{MH}$ or $\texttt{MH}$ class. 
2. For each model, create a joint proposal distribution for its parameters $\boldsymbol{\theta}$, where  each marginal distribution is normally distributed. 
2. For each model, select to sample 2000 realizations of the parameters from their joint distribution.

Print the posterior probability of each model.

In [ ]:
# Solution
from UQpy.SampleMethods import MH, MMH
proposals = [Normal(0,10),
            JointInd([Normal(0,1), Normal(0,1)]),
            JointInd([Normal(0,1), Normal(0,2), Normal(0.025)])]

selection = BayesModelSelection(
   data=data_ex1, candidate_models=[model1, model2, model3], prior_probabilities=[1./3., 1./3., 1./3.],
   nsamples=[2000, 2000, 2000], sampling_class=[MH, ]*3, jump=[1, 1, 1], nburn=[500, 500, 500], proposal=proposals,
   seed=[[0.], [0., 0.], [0., 0., 0.]])

### Activites

Change the parameter of the MCMC sampling in order to see how the results change.

In [ ]:
sorted_indices=np.argsort(selection.probabilities[::-1])
print('Posterior probabilities of sorted models:')
print([selection.probabilities[i] for i in sorted_indices])

# Exercise 3 


Learning the parameters of the quadratic model. Consider we have identified the quadratic model

$$m_2(\boldsymbol{\theta})=\theta_{0} x + \theta_{1} x^{2}$$

as the model that represents the data. Using the measurements, we want to learn the parameters $\theta_{0}$ and $\theta_{1}$ of the quadratic model.

In [ ]:
# Solution
data_ex1a = np.loadtxt('data_ex1a.txt')

### Step 1:

Import the necessary libraries.

1. From the $\texttt{UQpy.Inference}$ module import:
- $\texttt{BayesParameterEstimation}$

2. From the $\texttt{UQpy.Distributions}$ module import:
- $\texttt{Normal}$
- $\texttt{JoinInd}$ 

3. From $\texttt{UQpy.RunModel}$ module import $\texttt{RunModel}$.

In [ ]:
# Solution
from UQpy.Inference import BayesParameterEstimation
from UQpy.Distributions import Normal, JointInd

### Step 2:

Create instances of the $\texttt{RunModel}$ class for the quadratic model.

In [ ]:
# Solution
runmodel_quadratic = RunModel(model_script='pfn1.py', model_object_name='model_quadratic', vec=False,var_names=['theta_0','theta_1'])


### Step 3:

Create a joint distribution object for its parameters $\boldsymbol{\theta}$ that represents our prior belief. Each marginal distribution is Normally distributed but, we don't provide any information about its parameters.

In [ ]:
# Solution
prior = JointInd([Normal(), Normal()])

### Step 4:

Create an instance of the $\texttt{InferenceModel}$ class. 

In [ ]:
# Solution
inference_model = InferenceModel(nparams=2, runmodel_object=runmodel5, prior=prior, error_covariance=np.ones(50), name='model_quadratic')

### Step 5:

Create a joint proposal distribution object for its parameters $\boldsymbol{\theta}$. Each marginal distribution is normally distributed, i.e., $\theta_0 \sim \mathcal{N}(0.0, 0.1)$ and  $\theta_1 \sim \mathcal{N}(0.0, 0.05)$.

In [ ]:
# Solution
proposal=JointInd([Normal(0.0, 0.1), Normal(0.0, 0.05)])

### Step 6:

Create an instance of the $\texttt{BayesParameterEstimation}$ class.  Select as sampling algorithm the $\texttt{MH}$ algorithm of the $\texttt{MCMC}$ class.

Print the posterior values of the two parameters that are stored in the $\texttt{samples}$ attribute of the $\texttt{BayesParameterEstimation}$ object.

In [ ]:
# Solution
bayes_estimator = BayesParameterEstimation(
   data=data_ex1a, inference_model=inference_model, sampling_class=MH, nsamples=500, jump=10, nburn=0, 
   proposal=proposal, seed=[0.5, 2.5], verbose=True)
s = bayes_estimator.sampler.samples

print(s)